In [3]:
! pip install yfinance statsmodels plotly backtesting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 3.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.graph_objects as go
import warnings
from statsmodels.tsa.stattools import adfuller, coint
from sklearn.linear_model import LinearRegression
from scipy.stats import norm
from backtesting import Backtest , Strategy
from plotly.subplots import make_subplots
%matplotlib inline
warnings.filterwarnings("ignore")

/usr/local/lib/python3.12/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


**A list of tickers representing futures commodities chosen for this project**

In [5]:
comodities_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'TSLA', 'META', 'BRK-A', 'JPM', 'V',
                      'PG', 'MA', 'HD', 'UNH', 'KO', 'PEP', 'DIS', 'NFLX', 'CMCSA', 'ADBE']

# **Data preprocessing and cleaning**

In [6]:
data = pd.DataFrame()
for i in comodities_tickers:
  data[i]=  yf.Ticker(i).history(start = '2015-01-01',
                           end = '2025-10-30',
                           interval = '1d',
                           actions = True,
                           auto_adjust = True).Close
data.head()

,AAPL,MSFT,GOOGL,AMZN,NVDA,TSLA,META,BRK-A,JPM,V,PG,MA,HD,UNH,KO,PEP,DIS,NFLX,CMCSA,ADBE
Date,,,,,,,,,,,,,,,,,,,,
2015-01-02 00:00:00-05:00,24.237551,39.933052,26.296133,15.4260,0.483038,14.620667,77.969330,223600.0,46.720936,61.340973,66.518349,80.105675,80.418716,85.048477,29.992590,68.548561,85.943520,4.984857,22.256113,72.339996
2015-01-05 00:00:00-05:00,23.554737,39.565838,25.795088,15.1095,0.474880,14.006000,76.717072,220980.0,45.270489,59.986954,66.202057,77.852440,78.731476,83.647614,29.992590,68.033195,84.687584,4.731143,21.802223,71.980003
2015-01-06 00:00:00-05:00,23.556963,38.985111,25.158482,14.7645,0.460482,14.085333,75.683441,220450.0,44.096664,59.600407,65.900536,77.684174,78.490433,83.478806,30.220352,67.517853,84.238388,4.650143,21.517817,70.529999
2015-01-07 00:00:00-05:00,23.887276,39.480438,25.084492,14.9210,0.459282,14.063333,75.683441,223480.0,44.163967,60.398933,66.246216,78.892578,81.180664,84.331184,30.597586,69.492134,85.100121,4.674286,21.498335,71.110001
2015-01-08 00:00:00-05:00,24.805082,40.641876,25.171886,15.0230,0.476559,14.041333,77.700989,226680.0,45.150864,61.209042,67.003761,80.119705,82.976723,88.356552,30.967670,70.755104,85.980179,4.778000,21.965858,72.919998


In [7]:
data.isnull().sum().tolist()
# Output là danh sách tổng các giá trị còn thiếu cho mỗi cột trên

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [8]:
for i in data.columns:
  if data[i].isnull().sum()/len(data[i])> 0.01:    # Xóa các cột có tỷ lệ phần trăm các giá trị bị thiếu vượt quá 1%
    del data[i]
data.interpolate(limit_direction="both",inplace=True)   # Fill các dữ liệu thiếu còn lại bằng cách sử dụng interploation (nội suy)
data.tail()

,AAPL,MSFT,GOOGL,AMZN,NVDA,TSLA,META,BRK-A,JPM,V,PG,MA,HD,UNH,KO,PEP,DIS,NFLX,CMCSA,ADBE
Date,,,,,,,,,,,,,,,,,,,,
2025-10-23 00:00:00-04:00,259.328583,520.559998,253.080002,221.089996,182.160004,448.980011,734.000000,735600.0,294.540009,345.276093,151.153000,573.770020,385.029999,360.450012,69.940002,151.509995,113.029999,111.359001,29.299999,354.119995
2025-10-24 00:00:00-04:00,262.565491,523.609985,259.920013,224.210007,186.259995,433.720001,738.359985,738500.0,300.440002,346.693298,152.490005,573.669983,386.679993,362.500000,69.709999,151.550003,111.680000,109.469002,29.280001,353.519989
2025-10-27 00:00:00-04:00,268.549652,531.520020,269.269989,226.970001,191.490005,452.420013,750.820007,732650.0,304.149994,347.132416,151.740005,572.359985,385.269989,365.980011,70.059998,152.630005,112.339996,109.456001,29.420000,357.799988
2025-10-28 00:00:00-04:00,268.739471,542.070007,267.470001,229.250000,201.029999,460.549988,751.440002,722495.0,305.359985,346.214233,151.369995,565.929993,385.750000,367.839996,70.160004,150.119995,111.650002,110.250000,29.280001,359.910004
2025-10-29 00:00:00-04:00,269.438812,541.549988,274.570007,230.300003,207.039993,461.510010,751.669983,712900.0,305.510010,340.605347,148.770004,554.580017,378.040009,355.260010,68.349998,146.160004,110.239998,110.041000,28.530001,337.859985


**Identification of non-stationary assets to fulfill the cointegration property requirement**

In [9]:
train_data , test_data = data[:round(len(data)*0.8)],data[round(len(data)*0.8):]
non_stationary = []
for i in train_data.columns:
  if adfuller(data[i])[1]> 0.01:
    non_stationary.append(i)
non_stationary

['AAPL',
 'MSFT',
 'GOOGL',
 'AMZN',
 'NVDA',
 'TSLA',
 'META',
 'BRK-A',
 'JPM',
 'V',
 'PG',
 'MA',
 'HD',
 'UNH',
 'KO',
 'PEP',
 'DIS',
 'NFLX',
 'CMCSA',
 'ADBE']

**Selection of pairs exhibiting cointegration**

In [10]:
pair_coint = pd.DataFrame({'asset1':[], 'asset2':[], 'score':[], 'pvalue':[]})
# creating dataframe to store pairs that are cointegrated
for y in non_stationary:
  for x in non_stationary:
    if y != x:
      score, pvalue, _ = coint(train_data[y],train_data[x])
      if pvalue <= 0.05:
        pair_coint.loc[len(pair_coint.index)] = [y , x, score, pvalue]

list1 = [sorted([i,y]) for i , y in zip(pair_coint.asset1, pair_coint.asset2)]
index = []
list2 = []
# the code below consist of removing duplicated pairs
for i in list1:
  if list1.count(i)>1:
    list1.remove(i)

for i,y in zip(np.array(list1)[:,0], np.array(list1)[:,1]):
  row = pair_coint[(pair_coint['asset1'] == i) & (pair_coint['asset2'] == y)].index.tolist()
  index.append(row)

for i in index:
  for y in i:
    list2.append(y)
new_pair = pair_coint.loc[list2]
new_pair

,asset1,asset2,score,pvalue
11,HD,MSFT,-3.962480,0.008117
2,GOOGL,TSLA,-3.746095,0.015970
0,MSFT,PG,-3.652556,0.021062
10,MA,V,-4.799198,0.000379
11,HD,MSFT,-3.962480,0.008117
3,GOOGL,HD,-3.674051,0.019781
13,HD,PG,-3.570695,0.026622
19,PEP,UNH,-3.370846,0.045693
17,KO,UNH,-3.920397,0.009295
19,PEP,UNH,-3.370846,0.045693


**We define a fucntion that calculate the spread between two cointegrated pairs using rolling**

In [11]:
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm

def spread(y, x):
    # Thêm hằng số Alpha vào biến X
    X_exog = sm.add_constant(train_data[x])

    # Chạy hồi quy trượt (Rolling OLS) với cửa sổ 60 ngày
    # Thay vì hồi quy trên toàn bộ dữ liệu một lúc
    model = RollingOLS(train_data[y], X_exog, window=60)
    rres = model.fit()
    params = rres.params

    # spread = y - (alpha + beta * x)
    # params['const'] là alpha, params[x] là beta
    spread_series = train_data[y] - (params['const'] + params[x] * train_data[x])

    return spread_series.dropna()

In [12]:
def eligible_pair(pair):
    drop_list = []

    for i, (x, y) in enumerate(zip(pair.asset1.tolist(), pair.asset2.tolist())):
        try:
            s = spread(x, y)

            # Chuyển về Series và loại bỏ NaN/Inf để tránh lỗi tính toán
            s = pd.Series(s).replace([np.inf, -np.inf], np.nan).dropna()

            if len(s) < 30:
                drop_list.append(i)
                continue

            adf_result = adfuller(s)
            p_value = adf_result[1]

            if p_value > 0.05:
                drop_list.append(i)

        except Exception as e:
            print(f"Error processing pair {x}-{y}: {e}")
            drop_list.append(i)

    return pair.drop(pair.index[drop_list])

eligible_pair = eligible_pair(new_pair)
eligible_pair

,asset1,asset2,score,pvalue
11,HD,MSFT,-3.962480,0.008117
2,GOOGL,TSLA,-3.746095,0.015970
0,MSFT,PG,-3.652556,0.021062
10,MA,V,-4.799198,0.000379
11,HD,MSFT,-3.962480,0.008117
3,GOOGL,HD,-3.674051,0.019781
13,HD,PG,-3.570695,0.026622
19,PEP,UNH,-3.370846,0.045693
17,KO,UNH,-3.920397,0.009295
19,PEP,UNH,-3.370846,0.045693


## Variance Ratio Test - Lo and MacKinlay:
The LO-Mackinlay test is used to evaluate the efficiency of a time series or determine if it follows a random walk process. we need to identify the minimum p-value and retrieves the corresponding maximum z-statistic, which represents the highest significance level for the LO-Mackinlay variance test ratio

In [13]:
def rolling_variance_ratio(df):
  spread = df.diff().dropna()
  variance = spread.var()
  max_t = 40
  variance_ratio_test = pd.DataFrame({'variance':[], 'variance_increments': [], 'variance_ratio': [], 'standard_eroor':[],'z_stat':[],'pvalue':[]})
  for t in range(2,max_t):
    v_t = spread.rolling(t).sum().var()
    variance_ratio = v_t/(t*variance) -1
    standard_error = np.sqrt(2*(2*t - 1)*(t-1)/(3*t*len(spread)))
    z_stat = variance_ratio/standard_error
    p_value = 2*(1- norm.cdf(abs(z_stat)))
    variance_ratio_test.loc[len(variance_ratio_test.index)] = [variance, v_t, variance_ratio, standard_error, z_stat, p_value]

  min_pvalue = min(variance_ratio_test['pvalue'])
  max_z_stat =  variance_ratio_test[variance_ratio_test['pvalue'] == min_pvalue]['z_stat'].values # retreving the minimum pvalue with the corresponding z_stat

  return max_z_stat

In [14]:
# If any of the maximum z-statistics are greater than or equal to 0 (indicating mean-reverting behavior)
def eligible(pair):
  list1 = []
  for i, (x,y) in enumerate(zip(pair.asset1.tolist(),pair.asset2.tolist())):
    max_z_stat = rolling_variance_ratio(spread(x,y))
    if  (max_z_stat >= 0).any():
      list1.append(i)
  return pair.drop(pair.index[list1])
eligible = eligible(eligible_pair)
eligible

,asset1,asset2,score,pvalue
11,HD,MSFT,-3.962480,0.008117
2,GOOGL,TSLA,-3.746095,0.015970
0,MSFT,PG,-3.652556,0.021062
10,MA,V,-4.799198,0.000379
11,HD,MSFT,-3.962480,0.008117
3,GOOGL,HD,-3.674051,0.019781
19,PEP,UNH,-3.370846,0.045693
17,KO,UNH,-3.920397,0.009295
19,PEP,UNH,-3.370846,0.045693
18,KO,PEP,-3.881250,0.010526


Matrix of eligible pairs of assets, exhibiting mean-reverting behavior

In [15]:
# Extract the relevant columns
matrix_data = eligible[['asset1', 'asset2', 'pvalue']]

# Create matrix plot
fig = go.Figure(data=go.Scatter(
    x=matrix_data['asset2'],
    y=matrix_data['asset1'],   #
    mode='markers',
    marker=dict(
        color=matrix_data['pvalue'],
        colorscale='Viridis',
        size=20,
        colorbar=dict(title='P-value')
    ),
    text=matrix_data['pvalue'],
    hovertemplate='<b>Asset 1</b>: %{y}<br><b>Asset 2</b>: %{x}<br><b>P-value</b>: %{text:.4f}',
))

# Set axis labels
fig.update_xaxes(title_text='Asset 2')
fig.update_yaxes(title_text='Asset 1')

# Show figure
fig.show()

We retreive pairs that demonstrate a fundamental relationship, which will enable us to conduct backtesting

In [54]:
list_pairs = [['MA','V'],['ADBE','AMZN'],['KO','PEP'],['GOOGL','TSLA']]

In [55]:
#  calculates the z-score of the spread between two pairs
def zscore_spread(y,x):
  spread_mavg = spread(y,x)
  spread_mavg60 = spread(y,x).rolling(window=60, center=False).mean()
  spread_std_60 = spread(y,x).rolling(window=60, center=False).std()
  zscore_60 = (spread_mavg - spread_mavg60)/spread_std_60
  return zscore_60

In [56]:
# Calculate the threshold values
def get_thresholds(data):
    std_dev = np.std(data)
    threshold_upper = np.mean(data) + 1.7*std_dev
    threshold_lower = np.mean(data) - 1.7*std_dev
    return threshold_upper, threshold_lower

We generate trading signals for two pairs of assets. In this strategy we need to make sure that two pairs move differently in the same time. so if the spread exceeds the upper threshold and the corresponding asset's price has increased over the past three periods, a short position (-1) is assigned to that asset, and vice versa.

In [57]:
def signal(new_data, y_ticker, x_ticker):
    """
    Hàm sinh tín hiệu nhận vào DataFrame và tên 2 mã chứng khoán.
    """
    df = new_data.copy()
    threshold = 1.7

    # Logic Momentum: Giá hiện tại so với 3 ngày trước
    # Dùng đúng tên ticker được truyền vào
    y_momentum_up = df[y_ticker] > df[y_ticker].shift(3)
    y_momentum_down = df[y_ticker] < df[y_ticker].shift(3)

    # --- Tín hiệu ---
    # Short Spread (Bán Y): Z > 1.7 và Y đang tăng
    cond_short_y = (df['zscore'] > threshold) & y_momentum_up

    # Long Spread (Mua Y): Z < -1.7 và Y đang giảm
    cond_long_y = (df['zscore'] < -threshold) & y_momentum_down

    conditions = [cond_long_y, cond_short_y]
    choices = [1, -1] # 1: Buy, -1: Sell

    df['position_y'] = np.select(conditions, choices, default=0)
    df['position_x'] = -df['position_y'] # Hedge ngược chiều

    return df

In [58]:
def visualize_pair(y, x):
    # --- PHẦN 1: CHUẨN BỊ DỮ LIỆU (Giữ logic đúng để không bị lỗi code) ---
    # 1. Tính toán Spread và Z-score
    spread_series = spread(y, x)
    window = 60
    zscore = (spread_series - spread_series.rolling(window).mean()) / spread_series.rolling(window).std()

    # 2. Tạo DataFrame tổng hợp
    df_input = pd.DataFrame(index=spread_series.index)
    df_input[y] = train_data[y]
    df_input[x] = train_data[x]
    df_input['zscore'] = zscore
    df_input = df_input.dropna()

    # 3. Gọi hàm signal (Truyền đủ 3 tham số)
    data = signal(df_input, y, x)

    # Thiết lập ngưỡng (như mẫu của bạn)
    threshold_upper = 1.7
    threshold_lower = -1.7

    # --- PHẦN 2: VẼ BIỂU ĐỒ (Giống hệt mẫu bạn yêu cầu) ---
    # Layout: 3 hàng (Row 1: Y, Row 2: Spread, Row 3: X)
    fig_pair = make_subplots(rows=3, cols=1, vertical_spacing=0.025, row_heights=[1, 1.4, 1],
                             subplot_titles=(f'Price: {y}', 'Spread Z-Score', f'Price: {x}'))

    # === ROW 1: Cổ phiếu Y ===
    fig_pair.add_trace(go.Scatter(x=data.index, y=data[y], name=y), row=1, col=1)

    # Điểm Mua Y
    fig_pair.add_trace(go.Scatter(mode='markers', name=f'Buy {y}',
                                  x=data[data['position_y'] == 1].index,
                                  y=data[data['position_y'] == 1][y],
                                  marker=dict(color='green', size=10, symbol='triangle-up')), row=1, col=1)
    # Điểm Bán Y
    fig_pair.add_trace(go.Scatter(mode='markers', name=f'Sell {y}',
                                  x=data[data['position_y'] == -1].index,
                                  y=data[data['position_y'] == -1][y],
                                  marker=dict(color='red', size=10, symbol='triangle-down')), row=1, col=1)

    # === ROW 3: Cổ phiếu X ===
    fig_pair.add_trace(go.Scatter(x=data.index, y=data[x], name=x), row=3, col=1)

    # Điểm Mua X
    fig_pair.add_trace(go.Scatter(mode='markers', name=f'Buy {x}',
                                  x=data[data['position_x'] == 1].index,
                                  y=data[data['position_x'] == 1][x],
                                  marker=dict(color='green', size=10, symbol='triangle-up')), row=3, col=1)
    # Điểm Bán X
    fig_pair.add_trace(go.Scatter(mode='markers', name=f'Sell {x}',
                                  x=data[data['position_x'] == -1].index,
                                  y=data[data['position_x'] == -1][x],
                                  marker=dict(color='red', size=10, symbol='triangle-down')), row=3, col=1)

    # === ROW 2: Spread (Z-Score) ===
    # Đường Z-Score
    fig_pair.add_trace(go.Scatter(x=data.index, y=data['zscore'], mode='lines', name='Spread (Z-score)',
                                  line=dict(color='skyblue', dash='dashdot')), row=2, col=1)

    # Đường trung bình (Mean = 0)
    fig_pair.add_trace(go.Scatter(x=data.index, y=[0] * len(data), mode='lines', name='Mean',
                                  line=dict(color='black', dash='dash')), row=2, col=1)

    # Ngưỡng trên (Upper Threshold)
    fig_pair.add_trace(go.Scatter(x=data.index, y=[threshold_upper] * len(data), mode='lines',
                                  name='Upper Threshold', line=dict(color='red', dash='dash')), row=2, col=1)

    # Ngưỡng dưới (Lower Threshold)
    fig_pair.add_trace(go.Scatter(x=data.index, y=[threshold_lower] * len(data), mode='lines',
                                  name='Lower Threshold', line=dict(color='green', dash='dash')), row=2, col=1)

    # Update Layout
    fig_pair.update_layout(title=f'Pair Trading Strategy: {y} and {x}',
                           hovermode='x unified', title_x=0.45, height=900, showlegend=True)

    fig_pair.show()

In [59]:
#  Present Graph of each pair
for i in list_pairs:
  visualize_pair(i[0], i[1])

# Backtesting Strategy

In [61]:
class generating_signal:
  def __init__(self,first_pair, second_pair,test_data):
    self.first_pair = first_pair
    self.second_pair = second_pair
    self.test_data = test_data
    self.test_data1 = yf.Ticker(self.first_pair).history(period = '2y',
                           interval = '1d',
                           actions = True,
                           auto_adjust = True)
    self.test_data2 = yf.Ticker(self.second_pair).history(period = '2y',
                           interval = '1d',
                           actions = True,
                           auto_adjust = True)

    self.test_data.index = pd.to_datetime(self.test_data.index,format="%Y-%m-%d",utc=True)
    self.test_data1.index = pd.to_datetime(self.test_data1.index,format="%Y-%m-%d",utc=True)
    self.test_data2.index = pd.to_datetime(self.test_data2.index,format="%Y-%m-%d",utc=True)
    self.test_data1 = self.test_data1.loc[self.test_data.index[0]:]
    self.test_data2 = self.test_data2.loc[self.test_data.index[0]:]
    self.common = self.test_data1.index.intersection(self.test_data2.index)
    self.test_data1 = self.test_data1.loc[self.common]
    self.test_data2 = self.test_data2.loc[self.common]
    if True:
      self.signal()

  def pair_spread(self):
    # Add constant to the second asset's close prices
    X_exog = sm.add_constant(self.test_data2.Close)

    # Perform Rolling OLS regression with a window of 60
    # y = self.test_data1.Close, X = X_exog (self.test_data2.Close with constant)
    model = RollingOLS(self.test_data1.Close, X_exog, window=60)
    rres = model.fit()
    params = rres.params

    # Calculate spread = Y - (alpha + beta * X)
    # params['const'] is alpha, params['Close'] is beta for self.test_data2.Close
    # Need to handle potential NaN values from rolling window
    self.spread = self.test_data1.Close - (params['const'] + params['Close'] * self.test_data2.Close)
    return self.spread

  def zscore_spread(self):
    self.spread_mavg = self.pair_spread()
    self.spread_mavg60 = self.spread_mavg.rolling(window=60, center=False).mean()
    self.spread_std_60 = self.spread_mavg.rolling(window=60, center=False).std()
    self.zscore_60 = (self.spread_mavg - self.spread_mavg60)/self.spread_std_60
    return self.zscore_60.dropna()

  def get_thresholds(self):
    zscore = self.zscore_spread()
    self.std_dev = np.std(zscore)
    self.threshold_upper = np.mean(zscore) + 1.7*self.std_dev
    self.threshold_lower = np.mean(zscore) - 1.7*self.std_dev
    return self.threshold_upper, self.threshold_lower

  def signal(self):

    self.spread_data = self.zscore_spread()
    self.threshold_upper, self.threshold_lower = self.get_thresholds()

    # Align test_data1 and test_data2 to the index of spread_data
    aligned_test_data1_close = self.test_data1.Close.loc[self.spread_data.index]
    aligned_test_data2_close = self.test_data2.Close.loc[self.spread_data.index]

    self.test_data1['position_y'] = 0
    self.test_data2['position_x'] = 0

    # Use iterrows or iterate over the aligned index to avoid direct integer indexing issues
    for i, date in enumerate(self.spread_data.index):
      if i == 0: # Skip the first element for comparison with i-1
          continue

      # Access data using the aligned Series
      current_spread_zscore = self.spread_data.iloc[i]
      prev_test_data1_close = aligned_test_data1_close.iloc[i-1]
      current_test_data1_close = aligned_test_data1_close.iloc[i]
      prev_test_data2_close = aligned_test_data2_close.iloc[i-1]
      current_test_data2_close = aligned_test_data2_close.iloc[i]

      if (current_spread_zscore >= self.threshold_upper) and (current_test_data1_close > prev_test_data1_close):
        self.test_data1.loc[date, 'position_y'] = -1
      if (current_spread_zscore <= self.threshold_lower) and (current_test_data1_close < prev_test_data1_close):
        self.test_data1.loc[date, 'position_y'] = 1

      if (current_spread_zscore <= self.threshold_lower) and (current_test_data2_close > prev_test_data2_close) :
        self.test_data2.loc[date, 'position_x'] = -1
      if (current_spread_zscore >= self.threshold_upper) and (current_test_data2_close < prev_test_data2_close):
        self.test_data2.loc[date, 'position_x'] = 1

## Backtest first asset



In [63]:
asset1 = generating_signal('MA','V',test_data).test_data1
def SIGNAL1():
  return asset1.position_y
def SINGAL2():
  return asset1.position_x

In [64]:
class  Pairstrading(Strategy):
  def init(self):
    super().init()
    self.signal = self.I(SIGNAL1)
    self.cooldown = 0

  def next(self):
    super().next()
    price = self.data.Close[-1]

    if self.signal == 1 and self.cooldown == 0:
      self.buy(tp = 1.2*price , sl = 0.95*price, size = 0.1)
      self.cooldown = 3   # new trades can be taken only after 3 bars

    elif self.signal== -1 and self.cooldown == 0:
      self.sell(tp = 0.8*price , sl = 1.05*price, size = 0.1)
      self.cooldown = 3  # new trades can be taken only after 3 bars

    if self.cooldown > 0 :
      self.cooldown -= 1

In [65]:
bt = Backtest(asset1,Pairstrading,cash=100000,commission=0.001, exclusive_orders=True)
stat = bt.run()
evaluation = pd.DataFrame(stat)
evaluation

Backtest.run:   0%|          | 0/500 [00:00<?, ?bar/s]

,0
Start,2023-11-20 05:00:00+00:00
End,2025-11-18 05:00:00+00:00
Duration,729 days 00:00:00
Exposure Time [%],46.307385
Equity Final [$],102330.988842
Equity Peak [$],103135.474435
Commissions [$],238.154611
Return [%],2.330989
Buy & Hold Return [%],31.968494
Return (Ann.) [%],1.16576


In [66]:
bt.plot()

GridPlot(id='p2109', ...)

## Backtest Second asset

In [75]:
asset2 = generating_signal('MA','V',test_data).test_data2
def SIGNAL1():
  return asset2.position_y
def SIGNAL2():
  return asset2.position_x

In [76]:
class  Pairstrading(Strategy):
  def init(self):
    super().init()
    self.signal = self.I(SIGNAL2) # for second assets
    self.cooldown = 0

  def next(self):
    super().next()
    price = self.data.Close[-1]

    if self.signal == 1 and self.cooldown == 0:
      self.buy(tp = 1.2*price , sl = 0.95*price, size = 0.1)
      self.cooldown = 3   # new trades can be taken only after 3 bars

    elif self.signal== -1 and self.cooldown == 0:
      self.sell(tp = 0.8*price , sl = 1.05*price, size = 0.1)
      self.cooldown = 3  # new trades can be taken only after 3 bars

    if self.cooldown > 0 :
      self.cooldown -= 1

In [77]:
bt = Backtest(asset2,Pairstrading,cash=100000,commission=0.001, exclusive_orders=True)
stat = bt.run()
evaluation = pd.DataFrame(stat)
evaluation

Backtest.run:   0%|          | 0/500 [00:00<?, ?bar/s]

,0
Start,2023-11-20 05:00:00+00:00
End,2025-11-18 05:00:00+00:00
Duration,729 days 00:00:00
Exposure Time [%],4.191617
Equity Final [$],98241.440229
Equity Peak [$],100000.0
Commissions [$],78.971358
Return [%],-1.75856
Buy & Hold Return [%],30.3891
Return (Ann.) [%],-0.888445


In [78]:
bt.plot()

GridPlot(id='p2861', ...)